# ---Day 13: Care Package---

## ---Part One---

As you ponder the solitude of space and the ever-increasing three-hour roundtrip for messages between you and Earth, you notice that the Space Mail Indicator Light is blinking. To help keep you sane, the Elves have sent you a care package.

It's a new game for the ship's <a src="https://en.wikipedia.org/wiki/Arcade_cabinet">arcade cabinet</a>! Unfortunately, the arcade is ***all the way*** on the other end of the ship. Surely, it won't be hard to build your own - the care package even comes with schematics.

The arcade cabinet runs Intcode software (day9) like the game the Elves sent (your puzzle input). It has a primitive screen capable of drawing square ***tiles*** on a grid. The software draws tiles to the screen with output instructions: every three output instructions specify the `x` position (distance from the left), `y` position (distance from the top), and `tile id`. The `tile id` is interpreted as follows:

* 0 is an ***empty*** tile. No game object appears in this tile.
* 1 is a ***wall*** tile. Walls are indestructible barriers.
* 2 is a ***block*** tile. Blocks can be broken by the ball.
* 3 is a ***horizontal paddle*** tile. The paddle is indestructible.
* 4 is a ***ball*** tile. The ball moves diagonally and bounces off objects.

For example, a sequence of output values like `1,2,3,6,5,4` would draw a ***horizontal paddle*** tile (`1` tile from the left and `2` tiles from the top) and a ***ball*** tile (`6` tiles from the left and `5` tiles from the top).

Start the game. **How many block tiles are on the screen when the game exits?**

### Intcode Computer from Day09

In [1]:
class Computer:
    def __init__(self, program, input_val=1):
        self.program = program + [0 for i in range(1000)]
        self.input = input_val
        self.pointer = 0
        self.instruction = 0
        self.base = 0
        self.out = 0
        self.opcode = { 1 : self.add,
                       2 : self.multiply,
                       3 : self.save,
                       4 : self.output,
                       5 : self.jump_true,
                       6 : self.jump_false,
                       7 : self.less_than,
                       8 : self.is_equal,
                       9 : self.change_base,
                       99 : self.halt }
    
    def set_instruction(self):
        self.instruction = self.program[self.pointer]
        self.pointer += 1
        
    def get_opcode(self):
        return self.instruction % 100
    
    def get_modes(self):
        return self.instruction // 100

    def get_parameters(self, modes, num_parameters=3):
        mode = [ (modes // 10**n % 10) for n in range(num_parameters) ]
        parameters = []
        for i in range(self.pointer, self.pointer+num_parameters):
            if mode[i-self.pointer]:
                if mode[i-self.pointer] == 2:
                    parameters.append(self.program[self.program[i]+self.base])
                else:
                    parameters.append(self.program[i])
            else:
                parameters.append(self.program[self.program[i]])
        
        return parameters
    
    def set_write(self, modes, num_parameters):
        relative = [ (modes // 10**n % 10) for n in range(num_parameters) ][-1]
        if relative:
            return self.program[self.pointer+num_parameters-1]+self.base
        return self.program[self.pointer+num_parameters-1]

    def add(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] + values[1]
        self.pointer += step
        
    def multiply(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = values[0] * values[1]
        self.pointer += step
        
    def save(self, step=1):
        if self.get_modes():
            self.program[self.program[self.pointer]+self.base] = self.input
        else:
            self.program[self.program[self.pointer]] = self.input
        self.pointer += step
        
    def output(self, step=1):
        self.out = self.get_parameters(self.get_modes(), step)[0]
        print(f'Output: {self.out}')
        self.pointer += step 
        
    def jump_true(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] != 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def jump_false(self, step=2):
        values = self.get_parameters(self.get_modes(), step)
        if values[0] == 0:
            self.pointer = values[1]
        else:
            self.pointer += step
            
    def less_than(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = int(values[0] < values[1])
        self.pointer += step
    
    def is_equal(self, step=3):
        values = self.get_parameters(self.get_modes(), step)
        # Write over last parameter because it should always be a value indicating the location
        values[-1] = self.set_write(self.get_modes(), step)
        self.program[values[-1]] = int(values[0]==values[1])
        self.pointer += step
    
    def change_base(self, step=1):
        value = self.get_parameters(self.get_modes(), step)
        self.base += value[0]
        self.pointer += step
        
    def halt(self):
        print("Program Halted.")
        self.instruction = -1
    
    def run(self):
        while self.instruction != -1:
            #print(self.program, self.base)
            self.set_instruction()
            #print(self.pointer, self.instruction) # Used for Testing
            self.opcode[self.get_opcode()]()
            #print(self.program[:self.pointer+1], self.program[self.pointer:self.pointer+4]) # Used for Testing

In [2]:
class Arcade(Computer):
    def __init__(self, program,quarters=2):
        Computer.__init__(self, program)
        self.program[0] = quarters
        self.out_pointer = 0
        self.out = [0,0,0]
        self.score = 0
        self.objects = {0: [],\
                       1:[],\
                       2:[],\
                       3:[],\
                       4:[] } #empty, wall, block, paddle, ball
        
    def output(self, step=1):
        self.out[self.out_pointer] = self.get_parameters(self.get_modes(), step)[0]
        if self.out_pointer == 2:
            if self.out[0:2] == [-1,0]:
                self.score = self.out[-1]
            else:
                self.objects[self.out[2]].append(self.out[0:2])
        self.out_pointer = (self.out_pointer + 1) % 3
        self.pointer += step
        
    def save(self, step=1):
        self.input = self.joystick()
        if self.get_modes():
            self.program[self.program[self.pointer]+self.base] = self.input
        else:
            self.program[self.program[self.pointer]] = self.input
        self.pointer += step
        
    def joystick(self):
        if self.objects[4][-1][0] < self.objects[3][-1][0]:
            self.objects[3].append([self.objects[3][-1][0] - 1, self.objects[3][-1][1]])
            return -1
        elif self.objects[4][-1][0] > self.objects[3][-1][0]:
            self.objects[3].append([self.objects[3][-1][0] + 1,  self.objects[3][-1][1]])
            return 1
        else:
            return 0
        
    def visualize(self):
        key = [' ', '|', '#', '_', 'O']
        grid = [[ '' for i in range(37) ] for j in range(26)]
        while self.instruction != -1:
            self.set_instruction()
            self.opcode[self.get_opcode()]()
            if self.out[0:2] != [-1,0]:
                grid[self.out[1]][self.out[0]] = key[self.out[2]]
                rows = [ ''.join(line) for line in grid ]
                show = '\n'.join(rows)
                clear_output(wait=True)
                print(f'{show}', end ='\r' + f'Score : {self.score}', flush=True)
                #sleep(0.1)

In [3]:
with open('day13.txt') as f:
    game = [ int(i) for i in f.read().split(',') ]
    bb = Arcade(game,quarters=1)
    bb.run()
    print(f'There are {len(bb.objects[2])} blocks')

Program Halted.
There are 369 blocks


The set intersection of the blocks and the empty spaces equals the blocks left on the board.

In [4]:
len({tuple(i) for i in bb.objects[2]} - {tuple(j) for j in bb.objects[0]})

369

## ---Part Two---

The game didn't run because you didn't put in any quarters. Unfortunately, you did not bring any quarters. Memory address `0` represents the number of quarters that have been inserted; set it to `2` to play for free.

The arcade cabinet has a <a src ='https://en.wikipedia.org/wiki/Joystick'>joystick</a> that can move left and right. The software reads the position of the joystick with input instructions:

* If the joystick is in the ***neutral position***, provide `0`.
* If the joystick is ***tilted to the left***, provide `-1`.
* If the joystick is ***tilted to the right***, provide `1`.

The arcade cabinet also has a <a src='https://en.wikipedia.org/wiki/Display_device#Segment_displays'>segment display</a> capable of showing a single number that represents the player's current score. When three output instructions specify `X=-1, Y=0`, the third output instruction is not a tile; the value instead specifies the new score to show in the segment display. For example, a sequence of output values like `-1,0,12345` would show `12345` as the player's current score.

**Beat the game by breaking all the blocks. What is your score after the last block is broken?**

In [5]:
with open('day13.txt') as f:
    game = [ int(i) for i in f.read().split(',') ]
    bb = Arcade(game,quarters=2)
    bb.run()
    blocks = set(tuple(i) for i in bb.objects[2]) - set(tuple(j) for j in bb.objects[0])
    if blocks == set():
        blocks = 0
    print(f'You scored {bb.score}')
    print(f'There are {blocks} blocks remaining')

Program Halted.
You scored 19210
There are 0 blocks remaining


The set intersecion is an empty set because while playing the game, the program ouput [x,y,0] for all (x,y) in the list of blocks, thus we have broken all the blocks 

## Visualizer

A visualizer of this would be cool, I started making one but haven't finished yet. We'll see if I come back to it.